# Regime Detection Demo\n\nThis notebook demonstrates the core functionality of the QTFinance Markov-Switching Regime Trading System.

In [ ]:
# Import required libraries\nimport sys\nsys.path.append('..')\n\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Import QTFinance modules\nfrom src.data.loaders import YFinanceLoader\nfrom src.data.features import FeatureEngineer\nfrom src.data.validation import DataValidator\nfrom src.regime_detection.hmm_models import RegimeDetector\nfrom src.regime_detection.regime_filters import RegimeFilter\nfrom src.strategies.momentum import MomentumStrategy\nfrom src.utils.config import ConfigLoader\n\n# Set plotting style\nplt.style.use('seaborn-v0_8')\n%matplotlib inline

## 1. Load Configuration

In [ ]:
# Load configuration\nconfig_loader = ConfigLoader('../configs')\nconfig = config_loader.load_config('base_config.yaml')\n\nprint(\"Configuration loaded successfully!\")\nprint(f\"Number of regimes: {config['regime_detection']['n_regimes']}\")\nprint(f\"Min regime duration: {config['regime_detection']['min_regime_duration']} days\")

## 2. Fetch Market Data

In [ ]:
# Initialize data loader\nloader = YFinanceLoader()\n\n# Fetch S&P 500 data\nsymbols = ['SPY']  # S&P 500 ETF\nstart_date = '2010-01-01'\nend_date = '2023-12-31'\n\nprint(f\"Fetching data for {symbols} from {start_date} to {end_date}...\")\ndata = loader.fetch_data(symbols, start_date, end_date)\n\nprint(f\"Data shape: {data.shape}\")\nprint(f\"Columns: {data.columns.tolist()}\")\ndata.head()

## 3. Validate Data Quality

In [ ]:
# Validate data\nvalidator = DataValidator()\nis_valid, issues = validator.validate_ohlcv(data)\n\nif is_valid:\n    print(\"✅ Data validation passed!\")\nelse:\n    print(\"❌ Data validation issues:\")\n    for issue in issues:\n        print(f\"  - {issue}\")\n        \n# Clean data if needed\nif not is_valid:\n    print(\"\\nCleaning data...\")\n    data = validator.clean_data(data)\n    print(\"Data cleaned successfully!\")

## 4. Feature Engineering

In [ ]:
# Engineer features for regime detection\nengineer = FeatureEngineer()\n\n# Extract features\nfeatures, feature_names = engineer.engineer_features(\n    data,\n    feature_groups=['price', 'statistical', 'regime']\n)\n\nprint(f\"Engineered {len(feature_names)} features:\")\nfor i, name in enumerate(feature_names[:10]):\n    print(f\"  {i+1}. {name}\")\nprint(\"  ...\")\n\n# Display feature correlation\nplt.figure(figsize=(12, 10))\nsns.heatmap(features.corr(), cmap='coolwarm', center=0, annot=False)\nplt.title('Feature Correlation Matrix')\nplt.tight_layout()\nplt.show()

## 5. Regime Detection

In [ ]:
# Initialize regime detector\ndetector = RegimeDetector(\n    primary_model='hmm',\n    n_regimes=3,\n    validation_model='markov'\n)\n\n# Fit the model\nprint(\"Fitting regime detection model...\")\ndetector.fit(data, features)\n\n# Predict regimes\nregimes = detector.predict(data, features)\nregime_probs = detector.predict_proba(data, features)\n\nprint(\"\\nRegime distribution:\")\nprint(regimes.value_counts().sort_index())\nprint(\"\\nRegime labels:\")\nprint(\"  0: Bear Market\")\nprint(\"  1: Neutral Market\")\nprint(\"  2: Bull Market\")

## 6. Apply Regime Filtering

In [ ]:
# Apply regime filtering to reduce whipsaws\nfilter = RegimeFilter(\n    min_regime_duration=5,\n    probability_threshold=0.7\n)\n\n# Filter regimes\nfiltered_regimes = filter.filter_regimes(regimes, regime_probs)\n\n# Compare before and after filtering\nprint(\"Regime changes before filtering:\", (regimes.diff() != 0).sum())\nprint(\"Regime changes after filtering:\", (filtered_regimes.diff() != 0).sum())

## 7. Visualize Regimes

In [ ]:
# Create regime visualization\nfig, axes = plt.subplots(3, 1, figsize=(15, 10), sharex=True)\n\n# Plot 1: Price with regime coloring\nax1 = axes[0]\nprices = data['Close']['SPY']\n\n# Color by regime\ncolors = {0: 'red', 1: 'gray', 2: 'green'}\nregime_names = {0: 'Bear', 1: 'Neutral', 2: 'Bull'}\n\nfor regime in [0, 1, 2]:\n    mask = filtered_regimes == regime\n    ax1.plot(prices[mask], 'o', color=colors[regime], alpha=0.3, markersize=2, label=regime_names[regime])\n\nax1.plot(prices, 'k-', linewidth=0.5, alpha=0.5)\nax1.set_ylabel('SPY Price ($)')\nax1.set_title('S&P 500 Price with Regime Classification')\nax1.legend()\nax1.grid(True, alpha=0.3)\n\n# Plot 2: Regime probabilities\nax2 = axes[1]\nregime_probs.plot(ax=ax2, alpha=0.7)\nax2.set_ylabel('Probability')\nax2.set_title('Regime Probabilities')\nax2.legend(['Bear', 'Neutral', 'Bull'])\nax2.grid(True, alpha=0.3)\n\n# Plot 3: Returns by regime\nax3 = axes[2]\nreturns = prices.pct_change()\n\nfor regime in [0, 1, 2]:\n    mask = filtered_regimes == regime\n    ax3.scatter(returns.index[mask], returns[mask], color=colors[regime], alpha=0.5, s=10, label=regime_names[regime])\n\nax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)\nax3.set_ylabel('Daily Returns')\nax3.set_xlabel('Date')\nax3.set_title('Daily Returns by Regime')\nax3.legend()\nax3.grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.show()

## 8. Regime Statistics

In [ ]:
# Calculate regime statistics\nfrom src.regime_detection.regime_filters import RegimePersistence\n\npersistence = RegimePersistence()\nregime_stats = persistence.calculate_persistence_stats(filtered_regimes)\n\n# Display statistics\nfor regime, stats in regime_stats.items():\n    if regime != 'overall':\n        print(f\"\\n{regime} Regime Statistics:\")\n        print(f\"  - Percentage of time: {stats['percentage']:.1f}%\")\n        print(f\"  - Average duration: {stats['mean_duration']:.1f} days\")\n        print(f\"  - Number of occurrences: {stats['count']}\")\n        \n        # Calculate returns statistics\n        regime_returns = returns[filtered_regimes == regime]\n        print(f\"  - Average daily return: {regime_returns.mean()*100:.3f}%\")\n        print(f\"  - Daily volatility: {regime_returns.std()*100:.3f}%\")\n        print(f\"  - Annualized Sharpe: {np.sqrt(252) * regime_returns.mean() / regime_returns.std():.2f}\")

## 9. Generate Trading Signals

In [ ]:
# Initialize momentum strategy\nmomentum = MomentumStrategy(config['strategies']['momentum'])\n\n# Generate signals\nsignals = momentum.generate_signals(data, filtered_regimes)\n\n# Calculate positions with risk management\ncapital = 100000  # $100k starting capital\npositions = momentum.calculate_positions(\n    signals, \n    capital, \n    risk_params=config['risk']\n)\n\nprint(\"Signal generation complete!\")\nprint(f\"Total buy signals: {(signals['momentum_signal'] > 0).sum()}\")\nprint(f\"Total sell signals: {(signals['momentum_signal'] < 0).sum()}\")

## 10. Performance Analysis

In [ ]:
# Calculate strategy performance\nstrategy_returns = returns * signals['momentum_position'].shift(1)\n\n# Calculate cumulative returns\ncum_returns = (1 + strategy_returns).cumprod()\ncum_market = (1 + returns).cumprod()\n\n# Plot cumulative returns\nplt.figure(figsize=(15, 6))\nplt.plot(cum_returns.index, cum_returns, label='Strategy', linewidth=2)\nplt.plot(cum_market.index, cum_market, label='Buy & Hold SPY', linewidth=2, alpha=0.7)\n\n# Shade regime periods\nfor regime in [0, 1, 2]:\n    mask = filtered_regimes == regime\n    plt.fill_between(cum_returns.index, 0, cum_returns.max()*1.1, \n                     where=mask, alpha=0.1, color=colors[regime])\n\nplt.ylabel('Cumulative Returns')\nplt.xlabel('Date')\nplt.title('Strategy Performance vs Buy & Hold')\nplt.legend()\nplt.grid(True, alpha=0.3)\nplt.show()\n\n# Calculate metrics\nmetrics = momentum.calculate_performance_metrics(returns, signals['momentum_position'], benchmark=returns)\n\nprint(\"\\nPerformance Metrics:\")\nfor metric, value in metrics.items():\n    if 'return' in metric:\n        print(f\"{metric}: {value*100:.2f}%\")\n    elif 'ratio' in metric:\n        print(f\"{metric}: {value:.2f}\")\n    else:\n        print(f\"{metric}: {value:.2f}\")

## Summary\n\nThis notebook demonstrated:\n1. Loading and validating market data\n2. Engineering features for regime detection\n3. Detecting market regimes using Hidden Markov Models\n4. Filtering regimes to reduce whipsaws\n5. Generating trading signals based on regimes\n6. Analyzing strategy performance\n\nThe regime-based approach helps adapt strategies to changing market conditions, potentially improving risk-adjusted returns.